<a href="https://colab.research.google.com/github/Felizlin94/ml-titanic-analysis/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%AD%98%E6%B4%BB%E9%A0%90%E6%B8%AC%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C_3_%E9%81%8B%E7%94%A8%E8%B3%87%E6%96%99%E8%88%87%E7%89%B9%E5%BE%B5%E5%84%AA%E5%8C%96%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Drop unnecessary columns
df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)


### Comparison of Missing Value Imputation Strategies
# Missing value imputation with constant (Scores after some OneHotEncoder conversion: Mean = 0.7845, Std = 0.0231)
# imputer_age_constant = SimpleImputer(strategy='constant', fill_value=-1)
# imputer_embarked_constant = SimpleImputer(strategy='constant', fill_value='Missing')
# df['Age'] = imputer_age_constant.fit_transform(df[['Age']])
# df['Embarked'] = imputer_embarked_constant.fit_transform(df[['Embarked']]).ravel()

# Missing value imputation with median (Scores after some OneHotEncoder conversion: Mean = 0.7857, Std = 0.0184)
# imputer_age_median = SimpleImputer(strategy='median')
# imputer_embarked_median = SimpleImputer(strategy='most_frequent')
# df['Age'] = imputer_age_median.fit_transform(df[['Age']])
# df['Embarked'] = imputer_embarked_median.fit_transform(df[['Embarked']]).ravel()

# Missing value imputation with mean (Scores after some OneHotEncoder conversion: Mean = 0.7868, Std = 0.0176)
imputer_age_mean = SimpleImputer(strategy='mean')
imputer_embarked_mean = SimpleImputer(strategy='most_frequent')
df['Age'] = imputer_age_mean.fit_transform(df[['Age']])
df['Embarked'] = imputer_embarked_mean.fit_transform(df[['Embarked']]).ravel()

# Drop rows with missing values directly (Scores after some OneHotEncoder conversion: Mean = 0.7851, Std = 0.0310)
# df = df.dropna()

### Feature Creating
# Clip the first English character of the Cabin value as a new categorical feature
df['Cabin_Category'] = df['Cabin'].str[0]

# Create Family-Size as a new numerical feature
df['Family_Size'] = df['SibSp'] + df['Parch'] + 1

df.drop(['Cabin', 'SibSp', 'Parch'], axis=1, inplace=True)


### Comparison of Converting Non-numeric Columns to LabelEncoder and OneHotEncoder
# LabelEncoder Method (Scores after mean imputation: Mean = 0.7867, Std = 0.0148)
# label_encoders = {}
# for column in ['Sex', 'Embarked', 'Cabin_Category']:
#     le = LabelEncoder()
#     df[column] = le.fit_transform(df[column].astype(str))
#     label_encoders[column] = le

# OneHotEncoder Method (Scores after mean imputation: Mean = 0.7924, Std = 0.0140)
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Cabin_Category'],  dtype=int)

df_train = df

columns_X = list(set(df.columns) - {'Survived'})
columns_y = ['Survived']

train_X = df_train[columns_X]
train_y = df_train[columns_y]


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)
import numpy as np
print(f"Scores: Mean = {np.mean(scores):.4f}, Std = {np.std(scores):.4f}")


   Survived  Pclass   Age     Fare  Family_Size  Sex_female  Sex_male  \
0         0       3  22.0   7.2500            2           0         1   
1         1       1  38.0  71.2833            2           1         0   
2         1       3  26.0   7.9250            1           1         0   
3         1       1  35.0  53.1000            2           1         0   
4         0       3  35.0   8.0500            1           0         1   

   Embarked_C  Embarked_Q  Embarked_S  Cabin_Category_A  Cabin_Category_B  \
0           0           0           1                 0                 0   
1           1           0           0                 0                 0   
2           0           0           1                 0                 0   
3           0           0           1                 0                 0   
4           0           0           1                 0                 0   

   Cabin_Category_C  Cabin_Category_D  Cabin_Category_E  Cabin_Category_F  \
0                 0  